In [12]:


import io

class Mat:
  """Implementação de uma matriz
  """
  def __init__(self, rows, cols):                                         
      """Construtor da classe

      Args:
          rows (int): _linhas da matrz
          cols (int): _colunas da matriz
      """
      self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]

  def numRows (self): return len(self.mat)

  def numCols (self): return len(self.mat[0])

  def __str__(self):
        """"Devolve a matriz como uma string
        """
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    
  def __repr__(self):
        return str(self)
    
  def __getitem__ (self, n):
        """Interface para a indexação []
        """
        return self.mat[n]


def subst(x,y):
  """Função que permite substituir valores de match e mismatch no alinhamento local

  Args:
    x (str): elemento da s1
    y (str): elemento da s2 
  Returns:
    int: valores de match(+4) e mismatch(-2)
    """  
                                                        
  return 4 if x == y else -2                                          

class NW:
  """Alinhamento global entre duas sequências usando valores de match e mismatch pré defenidos

  """
  def __init__(self, s1, s2, g):
    """Contrutor da classe NW

    Args:
        s1 (str): 1ª sequência
        s2 (str): 2ª sequência
        g (int): Gap Penalty
    """
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):
      self.mat[L + 1][0] = g * (L + 1)
      self.tr[L + 1][0]  = 'C'

    for C in range(len(s2)):
      self.mat[0][C + 1] = g * (C + 1)
      self.tr[0][C + 1]  = 'E'

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
  
  def align_score(self):
    """ Função que recebe os alinhamentos

    Retorna:
        int: O score máximo do alinhamento NW entre duas sequências 
    """
    return self.mat[len(self.s1)][len(self.s2)]

  def rebuild(self):
    """Função que implementa o processo de reconstrução e recuperação do 
    alinhamento ótimo entre duas sequências

    Retorna:
        str: Alinhamento Ótimo entre as duas sequências
    """
                                              
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),               # Diagonal
        'E' : ( 0, -1),               # Esquerda
        'C' : (-1,  0)                # Cima
    }

    while L > 0 or C > 0:
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        

      L += DL
      C += DC

    return S1, S2

  def __repr__(self):
    """Função que premite ver a representação das matrizes S e T

    Retorna:
        _type_: Matriz S e Matriz T
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print("\n Matriz S :")
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print('\n Matriz T :', file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()


In [14]:
def teste_global_align():
    align = NW("ACHDHGT","THHDACCGT",-2)
    print(align)
    print("\n Score do Alinhamento:", align.align_score())
    print("\n Alinhamento ótimo:",*align.rebuild(), sep="\n")  
teste_global_align()


 Matriz S :
    -   T   H   H   D   A   C   C   G   T
-   0  -2  -4  -6  -8 -10 -12 -14 -16 -18
A  -2  -2  -4  -6  -8  -4  -6  -8 -10 -12
C  -4  -4  -4  -6  -8  -6   0  -2  -4  -6
H  -6  -6   0   0  -2  -4  -2  -2  -4  -6
D  -8  -8  -2  -2   4   2   0  -2  -4  -6
H -10 -10  -4   2   2   2   0  -2  -4  -6
G -12 -12  -6   0   0   0   0  -2   2   0
T -14  -8  -8  -2  -2  -2  -2  -2   0   6

 Matriz T :
  - T H H D A C C G T
- 0 E E E E E E E E E
A C D D D D D E E E E
C C D D D D C D D E E
H C D D D E E C D D D
D C D C D D E E E D D
H C D D D C D D D D D
G C D C C D D D D D E
T C D C C D D D D C D


 Score do Alinhamento: 6

 Alinhamento ótimo:
ACHD--HGT
THHDACCGT


In [23]:
import io

class Mat:

  def __init__(self, rows, cols):
        """Implementação de uma matriz
        """                                          
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]

  def numRows (self): return len(self.mat)

  def numCols (self): return len(self.mat[0])

  def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)

  def __repr__(self):
        return str(self)
        
  def __getitem__ (self, n):
        return self.mat[n]


def subst(x,y):
  """Função que permite substituir valores de match e mismatch no alinhamento local

  Args:
    x (str): elemento da s1
    y (str): elemento da s2 
  Returns:
    int: valores de match(+4) e mismatch(-2)
    """                                                         
  return 4 if x == y else -2
                                        
class SW:
  """Alinhamento global entre duas sequências usando valores de match e mismatch pré defenidos

  """
  def __init__(self, s1, s2, g):
    """Contrutor da classe SW

    Args:
        s1 (str): 1ª sequência
        s2 (str): 2ª sequência
        g (int): Gap Penalty
    """
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):
      self.mat[L + 1][0] = g * (L + 1)
      self.tr[L + 1][0]  = 'C'

    for C in range(len(s2)):
      self.mat[0][C + 1] = g * (C + 1)
      self.tr[0][C + 1]  = 'E'

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   
            self.mat[L+1][C    ] + g,               
            self.mat[L  ][C + 1] + g,               
            0]
        dirs = "DEC-"

        self.mat[L + 1][C + 1] = max(possiveis)
        if self.mat[L + 1][C + 1] != 0:
          self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]
  

  def align_score(self): 
    """ Função que recebe os alinhamentos

    Retorna:
        int: O score máximo do alinhamento SW entre duas sequências 
    """
    align_score=0
    row = 0
    column = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if max(x1) > align_score:
          row = L
        column = C
      column -= 1
      return [align_score,row,column]

  def rebuild(self):
    """Função que implementa o processo de reconstrução e recuperação do 
    alinhamento ótimo entre duas sequências

    Retorna:
        str: Alinhamento Ótimo entre as duas sequências
    """                                          
    align_score=0
    row = 0
    column = 0
    for L, x1 in enumerate(self.mat):
      for C, x2 in enumerate(self.mat):
        if max(x1) > align_score:
          row = L
        column = C
      column -= 1
    L  = row
    C  = column
    S1 = ""
    s2 = ""   
    
    dirs = {
        'D' : (-1, -1),               
        'E' : ( 0, -1),               
        'C' : (-1,  0),
        '-' : (0, 0)               
    }

    while L <= 0 or C <= 0:
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        

      L += DL
      C += DC

    return S1, S2
  
  def __repr__(self):
    """Função que premite ver a representação das matrizes S e T

    Retorna:
        str: Matriz S e Matriz T
    """
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print("\n Matriz S :")
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print('\n Matriz T :', file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()


In [25]:
def teste_alignSW():
    align = SW("ACGT","TACCGT",-4)
    print(align)
    print("\n Score do Alinhamento:", align.align_score())
    print("\n Alinhamento ótimo:",*align.rebuild(), sep="\n")  
teste_alignSW()


 Matriz S :
    -   T   A   C   C   G   T
-   0  -4  -8 -12 -16 -20 -24
A  -4   0   0   0   0   0   0
C  -8   0   0   4   4   0   0
G -12   0   0   0   2   8   4
T -16   0   0   0   0   4  12

 Matriz T :
  - T A C C G T
- 0 E E E E E E
A C 0 0 0 0 0 0
C C 0 0 D D 0 0
G C 0 0 0 D D E
T C 0 0 0 0 C D


 Score do Alinhamento: [0, 0, 3]


UnboundLocalError: local variable 'S2' referenced before assignment